# Analyze Titanic data. 
I. Start with basic EDA (Exploratory data analysis): 

1. Compute average `Age` of passengers and number of passengers who survived and not survived grouped by `Sex` and `Passenger Class` (24 numbers);

2. What can you say about survivors based on the resulting table (open question), e.g. what is the surviving ratio for females in First class compared to the Second and Third? This answer is limited to 150 words.

3. What is the average number of males and females on all boats (rounded to the closest integer)? 

Do not forget to filter out all `?` in `Life Boat` attribute. 


In [1]:
import pandas as pd
import numpy as np

In [76]:
df = pd.read_csv("titanic_data.csv")
df.head()

,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Life Boat,Survived
0,First,"Allen, Miss. Elisabeth Walton",Female,29.0000,0,0,24160,211.3375,B5,Southampton,2,Yes
1,First,"Allison, Master. Hudson Trevor",Male,0.9167,1,2,113781,151.5500,C22 C26,Southampton,11,Yes
2,First,"Allison, Miss. Helen Loraine",Female,2.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,No
3,First,"Allison, Mr. Hudson Joshua Creighton",Male,30.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,No
4,First,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",Female,25.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,No


In [77]:
df.groupby(["Survived","Sex","Passenger Class"])["Age"].mean()

Survived  Sex     Passenger Class
No        Female  First              35.200000
                  Second             34.090909
                  Third              23.418750
          Male    First              43.658163
                  Second             33.092593
                  Third              26.679598
Yes       Female  First              37.109375
                  Second             26.711051
                  Third              20.814815
          Male    First              36.168240
                  Second             17.449274
                  Third              22.436441
Name: Age, dtype: float64

In [78]:
survived = df.groupby(["Survived","Sex","Passenger Class"])["Name"].count()
survived

Survived  Sex     Passenger Class
No        Female  First                5
                  Second              12
                  Third              110
          Male    First              118
                  Second             146
                  Third              418
Yes       Female  First              139
                  Second              94
                  Third              106
          Male    First               61
                  Second              25
                  Third               75
Name: Name, dtype: int64

In [79]:
print("Female survived ratio in ")
for ind in list(["First", "Second", "Third"]):
    print(ind + " class: " + str(survived.loc[("Yes","Female",ind)] / survived.loc[("No","Female",ind)]))

Female survived ratio in 
First class: 27.8
Second class: 7.833333333333333
Third class: 0.9636363636363636


In [80]:
survived[[1]]

Survived  Sex     Passenger Class
No        Female  Second             12
Name: Name, dtype: int64

In [81]:
df_onboat = df.dropna(axis='index', how='any', subset=['Life Boat'])
df_onboat['Life Boat'].unique()

array(['2', '11', '3', '10', 'D', '4', '9', '6', 'B', '8', 'A', '5', '7',
       'C', '14', '5 9', '13', '1', '15', '5 7', '8 10', '12', '16',
       '13 15 B', 'C D', '15 16', '13 15'], dtype=object)

In [82]:
female_onboat = df_onboat.groupby(["Sex","Life Boat"])["Name"].count().loc[("Female")].mean()
male_onboat = df_onboat.groupby(["Sex","Life Boat"])["Name"].count().loc[("Male")].mean()
print(round(female_onboat), round(male_onboat))

14 7


# II. Proceed with feature generation.
1. Drop the column `Life Boat`.
2. Generate new attribute `Family size` (sum up `No of Parents or Children on Board` and `No of Siblings or Spouses on Board`). What is the average family size? In which class did the biggest family travel?
Do not drop original attributes.
3. It seems that `Passenger Fare` is total among all passengers with the same `Ticket Number`: create new attribute `Single passenger fare`. For every passenger you need to compute the number of passengers with the same `Ticket Number` and then use this number as a divisor for `Passenger Fare`. 
Do not drop the original attribute.
4. Impute missing values: for numerical attributes use averaging over three groups: `Passenger Class`, `Sex`, `Embarkation Port`; for every numerical attribute create separate column that contain 1 for imputed value and 0 for originally presented. 
This step is mainly for practicing your groupby/join skills. In real tasks this kind of imputation is relatively rare.
5. Pre-process categorical attributes: For every categorical attribute create a separate column that contains 1 for imputed value and 0 for originally presented. One-hot encode categorical attributes with less than 20 unique values, drop other categorical attributes; drop original attributes. 
6. Set the role of the `Survived` attribute to `label`.


In [83]:
del df["Life Boat"]

In [84]:
df["Family size"] = df["No of Parents or Children on Board"] + df["No of Siblings or Spouses on Board"]

In [85]:
df["Family size"].mean()

0.8838808250572956

In [86]:
df[df["Family size"] == df["Family size"].max()]

,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Survived,Family size
1170,Third,"Sage, Master. Thomas Henry",Male,NaN,8,2,CA. 2343,69.55,NaN,Southampton,No,10
1171,Third,"Sage, Master. William Henry",Male,14.5,8,2,CA. 2343,69.55,NaN,Southampton,No,10
1172,Third,"Sage, Miss. Ada",Female,NaN,8,2,CA. 2343,69.55,NaN,Southampton,No,10
1173,Third,"Sage, Miss. Constance Gladys",Female,NaN,8,2,CA. 2343,69.55,NaN,Southampton,No,10
1174,Third,"Sage, Miss. Dorothy Edith 'Dolly'",Female,NaN,8,2,CA. 2343,69.55,NaN,Southampton,No,10
1175,Third,"Sage, Miss. Stella Anna",Female,NaN,8,2,CA. 2343,69.55,NaN,Southampton,No,10
1176,Third,"Sage, Mr. Douglas Bullen",Male,NaN,8,2,CA. 2343,69.55,NaN,Southampton,No,10
1177,Third,"Sage, Mr. Frederick",Male,NaN,8,2,CA. 2343,69.55,NaN,Southampton,No,10
1178,Third,"Sage, Mr. George John Jr",Male,NaN,8,2,CA. 2343,69.55,NaN,Southampton,No,10
1179,Third,"Sage, Mr. John George",Male,NaN,1,9,CA. 2343,69.55,NaN,Southampton,No,10


In [87]:
df_ticket_number = df['Ticket Number'].value_counts().to_frame().reset_index().rename(columns={'index':'Ticket Number', 'Ticket Number':'Ticket count'})
df = pd.merge(df, df_ticket_number, on='Ticket Number')
df["Single passenger fare"] = df["Passenger Fare"] / df["Ticket count"]
df

,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Survived,Family size,Ticket count,Single passenger fare
0,First,"Allen, Miss. Elisabeth Walton",Female,29.0000,0,0,24160,211.3375,B5,Southampton,Yes,0,4,52.834375
1,First,"Kreuchen, Miss. Emilie",Female,39.0000,0,0,24160,211.3375,NaN,Southampton,Yes,0,4,52.834375
2,First,"Madill, Miss. Georgette Alexandra",Female,15.0000,0,1,24160,211.3375,B5,Southampton,Yes,1,4,52.834375
3,First,"Robert, Mrs. Edward Scott (Elisabeth Walton Mc...",Female,43.0000,0,1,24160,211.3375,B3,Southampton,Yes,1,4,52.834375
4,First,"Allison, Master. Hudson Trevor",Male,0.9167,1,2,113781,151.5500,C22 C26,Southampton,Yes,3,6,25.258333
5,First,"Allison, Miss. Helen Loraine",Female,2.0000,1,2,113781,151.5500,C22 C26,Southampton,No,3,6,25.258333
6,First,"Allison, Mr. Hudson Joshua Creighton",Male,30.0000,1,2,113781,151.5500,C22 C26,Southampton,No,3,6,25.258333
7,First,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",Female,25.0000,1,2,113781,151.5500,C22 C26,Southampton,No,3,6,25.258333
8,First,"Cleaver, Miss. Alice",Female,22.0000,0,0,113781,151.5500,NaN,Southampton,Yes,0,6,25.258333
9,First,"Daniels, Miss. Sarah",Female,33.0000,0,0,113781,151.5500,NaN,Southampton,Yes,0,6,25.258333


In [105]:
#df.fillna(df.mean())
# col with nan values {'Cabin' - categorical, 'Port of Embarkation' - categorical, 
#'Single passenger fare' - num, 'Passenger Fare' - num, 'Age' - num}
# `Passenger Class`, `Sex`, `Embarkation Port`
if __name__ == '__main__':
    def nan_to_average(data):
        data_mean_val = data.groupby(["Passenger Class", "Sex", "Port of Embarkation"])["Single passenger fare"].mean()
        # sign for modification
        data["IsModified"] = 0
        for index, row in data.iterrows():
            for el in row:
                #if type(el) != str and el !="":
                if (row[row == el].index[0] == "Single passenger fare") and math.isnan(el):
                    data["IsModified"][index] = 1
                    data["Single passenger fare"][index] = data_mean_val.loc[(row.at["Passenger Class"],row.at["Sex"],row.at["Port of Embarkation"])]
        return data
    nan_to_average(df)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [90]:
df.groupby(["Passenger Class", "Sex", "Port of Embarkation"])["Single passenger fare"].mean()

Passenger Class  Sex     Port of Embarkation
First            Female  Cherbourg              38.668365
                         Queenstown             30.000000
                         Southampton            32.516189
                 Male    Cherbourg              37.856094
                         Queenstown             30.000000
                         Southampton            29.132124
Second           Female  Cherbourg              12.538476
                         Queenstown             12.350000
                         Southampton            11.273342
                 Male    Cherbourg              13.762104
                         Queenstown             11.489160
                         Southampton            11.130229
Third            Female  Cherbourg               6.292523
                         Queenstown              7.687132
                         Southampton             6.965251
                 Male    Cherbourg               7.087964
                         Qu

In [104]:
for index, row in df.iterrows():
            for el in row:
                print(row[row == "First"].index[0])
                #print(pd.Index(row).get_loc("First")) #Index(myseries).get_loc(7)
                break
            break

Passenger Class


In [95]:
type(2) != str

True

In [93]:
el

'First'